In [75]:
# Create Table
import uuid
import sqlalchemy as db
engine = db.create_engine('sqlite:///tideDB.sqlite3')
connection = engine.connect()
metadata = db.MetaData()

tide = db.Table('tidevals', metadata,
               db.Column('id', db.Integer, primary_key=True),
               db.Column('buoyid', db.Integer, nullable=False),
               db.Column('location', db.String(50), nullable=False),
               db.Column('datetime', db.String(255), nullable=False),
               db.Column('predicted', db.Float(), nullable=False),
               db.Column('highlow', db.String(2), nullable=False),
              sqlite_autoincrement=True)

metadata.create_all(engine)

In [9]:
import requests
import time
def initial_calls():
    """ 
    Makes calls for each buoy on list for year intervals on list.  Stores in one big array.
    Only about 130k records so one big array is fine.  Not sure point at which that needs to 
    be broken up.

    """
    KPList = list(historicInfill.values())[0]
    keyList = [list(i.keys())[0] for i in KPList]
    station_list_keys = list(tide_station_list.keys())
    station_list_values = list(tide_station_list.values())
    bigList = []
    # for j in range(len(station_list_keys)):
    for j in range(1):
        id = station_list_values[j]
        idName = station_list_keys[j]
        # for i in range(len(keyList)):
        for i in range(3):
            start = list(KPList[i].values())[0]['start']
            end = list(KPList[i].values())[0]['end']
            call = requests.get(f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date={start}&end_date={end}&station={id}&product=predictions&datum=STND&time_zone=gmt&interval=hilo&units=english&application=Waves_data&format=json")
            try:
                response = call.json()['predictions']
            except KeyError:
                print("Failed at {idName}".format)
                return bigList
            oneStep = [list(i.values()) for i in response]
            print(id, idName, start, end)
            [i.extend([id, idName]) for i in oneStep]
            bigList.extend(oneStep)
            if (i + 1) % 5 == 0:
                print("Buoy {} API call {} / {} complete, sleeping 45".format(id, i+1, len(keyList)))
                time.sleep(45)
        
        print(len(bigList))
        print(len(bigList[i]))
    return bigList

In [10]:
a = initial_calls()


9444900 PortTownsend_WA 19000101 19051231
9444900 PortTownsend_WA 19060101 19091231
9444900 PortTownsend_WA 19100101 19151231
22226
5


['1915-12-31 18:31', '12.622', 'H', 9444900, 'PortTownsend_WA']

In [76]:
import requests
a = tideTest()

data = []
for rows in a:
    entry = {'buoyid':rows[3], 'location':rows[4], 'datetime':rows[0], 'predicted':rows[1], 'highlow':rows[2]} 
    data.append(entry)

Buoy 9444900 API call 5 / 40 complete
Buoy 9444900 API call 10 / 40 complete
Buoy 9444900 API call 15 / 40 complete
Buoy 9444900 API call 20 / 40 complete
Buoy 9444900 API call 25 / 40 complete
Buoy 9444900 API call 30 / 40 complete
Buoy 9444900 API call 35 / 40 complete
Buoy 9444900 API call 40 / 40 complete
Buoy 9444090 API call 5 / 40 complete
Buoy 9444090 API call 10 / 40 complete
Buoy 9444090 API call 15 / 40 complete
Buoy 9444090 API call 20 / 40 complete
Buoy 9444090 API call 25 / 40 complete
Buoy 9444090 API call 30 / 40 complete
Buoy 9444090 API call 35 / 40 complete
Buoy 9444090 API call 40 / 40 complete


In [77]:
# To insert data into the db
query = db.insert(tide)
ResultProxy = connection.execute(query, data)

In [82]:
import pandas as pd
# To read out data from db
results = connection.execute(db.select([tide])).fetchall()
df = pd.DataFrame(results)
try:
    df.columns = results[0].keys()
except IndexError:
    print("Table is empty or doesn't exist")
df.head()

,id,buoyid,location,datetime,predicted,highlow
0,1,9444900,PortTownsend_WA,2095-01-01 02:23,5.218,L
1,2,9444900,PortTownsend_WA,2095-01-01 10:32,9.466,H
2,3,9444900,PortTownsend_WA,2095-01-01 13:07,9.124,L
3,4,9444900,PortTownsend_WA,2095-01-01 19:24,11.985,H
4,5,9444900,PortTownsend_WA,2095-01-02 02:52,4.198,L


In [74]:
# drop a table
emp.drop(engine)  # Drops a single table

metadata.drop_all(engine) # drops all tables in the database



In [70]:
len(h)

TypeError: object of type 'NoneType' has no len()

In [5]:
# #from flask import Flask
# #import openaq
# import requests

# response = requests.get('https://api.openaq.org/v1/measurements?city=Los%20Angeles&parameter=pm25')
# rj = response.json()
# rj = list(response.json().values())[1]
# rj

In [119]:
# def get_data():
#     response = requests.get(
#       "https://api.openaq.org/v1/measurements?city=Los%20Angeles&parameter=pm25")
#     return response
# API = get_data()


# def date_val(res):
#     """ extracts date & value """
#     rj = list(res.json().values())[1]
#     tupes = []
#     for i in rj:
#         a = (i['date']['utc'])
#         b = (i['value'])
#         tupelo = (a, b)
#         tupes.append(tupelo)
#     return tupes

# a = date_val(API)

In [6]:
# a

In [96]:
def date_val(res):
    """ extracts date & value """
    rj = list(res.json().values())[1]
    tupes = []
    for i in rj:
        a = (i['date']['utc'])
        b = (i['value'])
        tupelo = (a, b)
        tupes.append(tupelo)
    return tupes

date_val()

In [7]:
# """OpenAQ Air Quality Dashboard with Flask."""
# from flask import Flask
# from flask_sqlalchemy import SQLAlchemy
# import requests

# APP = Flask(__name__)
# APP.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///db.sqlite3'
# APP.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
# DB = SQLAlchemy(APP)


# def get_data():
#     response = requests.get(
#       """https://api.openaq.org/v1/
#       measurements?city=Los%20Angeles&parameter=pm25""")
#     return response
# API = get_data()


# def date_val(res):
#     """ extracts date & value """
#     rj = list(res.json().values())[1]
#     tupes = []
#     for i in rj:
#         a = (i['date']['utc'])
#         b = (i['value'])
#         tupelo = (a, b)
#         tupes.append(tupelo)
#     return tupes


# class Record(DB.Model):
#     """ DB model for data """
#     id = DB.Column(DB.Integer, primary_key=True)
#     datetime = DB.Column(DB.String(25))
#     value = DB.Column(DB.Float, nullable=False)

#     def __repr__(self):
#         return """<Time:{datetime},
#         Value:{value} >""".format(datetime=self.datetime, value=self.value)


# @APP.route('/')
# def root():
#     """ Base view. """
#     r = get_data()
#     dv = date_val(r)
#     check = Record.query.all()
#     if len(check) == 0:
#         for recs in dv:
#             db_record = Record(id=id(recs), datetime=recs[0], value=recs[1])
#             DB.session.add(db_record)
#             DB.session.commit()
#     out1 = str(dv)
#     return out1


# @APP.route('/get_results')
# def get_results():
#     """ Base view. """
#     response = get_data()
#     dd = date_val(response)
#     check = Record.query.all()
#     if len(check) == 0:
#         for recs in dd:
#             db_record = Record(id=id(recs), datetime=recs[0], value=recs[1])
#             DB.session.add(db_record)
#             DB.session.commit()
#     moTen = Record.query.filter(Record.value >= 10).all()
#     out2 = str(moTen)
#     return out2


# @APP.route('/refresh')
# def refresh():
#     """ Pull fresh data from Open AQ and replace
#     existing data. """
#     DB.drop_all()
#     DB.create_all()
#     upRes = get_data()
#     upDv = date_val(upRes)
#     for recs in upDv:
#         db_record = Record(id=id(recs), datetime=recs[0], value=recs[1])
#         DB.session.add(db_record)
#         DB.session.commit()
#     return 'Data refreshed!'



In [4]:
import datetime
import math
import requests

In [2]:
# # Function to split up input date into batches of 30 days
# # Managing different month day count will be annoying
# def batchAPI(date1, date2):
#     """
#     Inputs: date strings, format: "day/month/year"(older one), "day/month/year"(newer one)
#     Runtime = (numStations * numdays * numYears)*11 seconds (best case)
#     Outputs: One huge list of sets of days of data
#     """
#     dates = [date1, date2]
#     dt1, dt2 = [datetime.datetime.strptime(x, "%d/%m/%Y") for x in dates]
#     finalList = []  # Will contain n sets of 30 day api calls
#     dateRange = dt2 - dt1
#     dateRangeChunks = math.ceil(int(dateRange.days) / 30)
#     dateRangeChunk = []  # Need to make 30 minute intervals

#     return dateRangeChunks, dateRange

In [47]:
batchAPI('3/1/2020', '1/5/2021')

(17, datetime.timedelta(days=484))

In [3]:

fmt = "YrMoDa"
start = "19060101"
end = "19101231"
i = 9410170
response2 = requests.get(f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date={start}&end_date={end}&station={i}&product=predictions&datum=STND&time_zone=gmt&interval=hilo&units=english&application=Waves_data&format=json")


In [3]:
"""5 year intervals from 1900 - 2099"""
historicInfill = {
    "dates":[
    {"1900":{"start":"19000101", "end":"19051231" }},
    {"1906":{"start":"19060101", "end":"19091231" }},
    {"1910":{"start":"19100101", "end":"19151231" }},
    {"1916":{"start":"19160101", "end":"19191231" }},
    {"1920":{"start":"19200101", "end":"19251231" }},
    {"1926":{"start":"19260101", "end":"19291231" }},
    {"1930":{"start":"19300101", "end":"19351231" }},
    {"1936":{"start":"19360101", "end":"19391231" }},
    {"1940":{"start":"19400101", "end":"19451231" }}, 
    {"1946":{"start":"19460101", "end":"19491231" }},
    {"1950":{"start":"19500101", "end":"19551231" }},
    {"1956":{"start":"19560101", "end":"19591231" }},
    {"1960":{"start":"19600101", "end":"19651231" }},
    {"1966":{"start":"19660101", "end":"19691231" }},
    {"1970":{"start":"19700101", "end":"19751231" }},
    {"1976":{"start":"19761231", "end":"19791231" }},
    {"1980":{"start":"19800101", "end":"19851231" }},
    {"1986":{"start":"19860101", "end":"19891231" }},
    {"1990":{"start":"19900101", "end":"19951231" }},
    {"1996":{"start":"19960101", "end":"19991231" }},
    {"2000":{"start":"20000101", "end":"20051231" }},
    {"2005":{"start":"20050101", "end":"20091231" }},
    {"2010":{"start":"20100101", "end":"20151231" }},
    {"2016":{"start":"20160101", "end":"20191231" }},
    {"2020":{"start":"20200101", "end":"20251231" }},
    {"2026":{"start":"20260101", "end":"20291231" }},
    {"2030":{"start":"20300101", "end":"20351231" }},
    {"2036":{"start":"20360101", "end":"20391231" }},
    {"2040":{"start":"20400101", "end":"20451231" }},
    {"2045":{"start":"20450101", "end":"20491231" }},
    {"2050":{"start":"20500101", "end":"20551231" }},
    {"2056":{"start":"20560101", "end":"20591231" }},
    {"2060":{"start":"20600101", "end":"20651231" }},
    {"2065":{"start":"20650101", "end":"20691231" }},
    {"2070":{"start":"20750101", "end":"20751231" }},
    {"2076":{"start":"20760101", "end":"20791231" }},
    {"2080":{"start":"20800101", "end":"20851231" }},
    {"2086":{"start":"20860101", "end":"20891231" }},
    {"2090":{"start":"20900101", "end":"20951231" }},
    {"2095":{"start":"20950101", "end":"20991231" }}]
}

In [4]:
tide_station_list = {
    'PortTownsend_WA' : 9444900,
    'PortAngeles_WA' : 9444090,
    'NeahBay_WA' : 9443090,
    'LaPushQuillayuteRiver_WA' : 9442396,
    'Westport_WA' : 9441102,
    'TokePoint_WA' : 9440910,
    'CapeDisappointment_WA' : 9440581,
    'Skamokawa_WA' : 9440569,
    'Wauna_OR' : 9439099,
    'Astoria_OR' : 9439040,
    'Garibali_OR' : 9437540,
    'SouthBeach_OR' : 9435380,
    'Charleston_OR' : 9432780,
    'PortOrford_OR' : 9431647,
    'CrescentCity_CA' : 9419750,
    'NorthSpit_CA' : 9418767,
    'ArenaCove_CA' : 9416841,
    'PointReyes_CA' : 9415020,
    'PortChicago_CA' : 9415144,
    'SanFrancisco_CA' : 9414290,
    'RedwoodCity_CA' : 9414523,
    'Monterey_CA' : 9413450,
    'PortSanLuis_CA' : 9412110,
    'OilPlatfrom_CA' : 9411406,
    'SantaBarbara_CA' : 9411340,
    'SantaMonica_CA' : 9410840,
    'LosAngeles_CA' : 9410660,
    'LaJolla_CA' : 9410230,
    'SanDiego_CA' : 9410170,
}


In [94]:
import time
def initial_calls():
    """ 
    Makes calls for each buoy on list for year intervals on list.  Stores in one big array.
    Only about 130k records so one big array is fine.  Not sure point at which that needs to 
    be broken up.

    """
    KPList = list(historicInfill.values())[0]
    keyList = [list(i.keys())[0] for i in KPList]
    station_list_keys = list(tide_station_list.keys())
    station_list_values = list(tide_station_list.values())
    bigList = []
    for j in range(len(station_list_keys)):
        id = station_list_values[j]
        idName = station_list_keys[j]
        for i in range(len(keyList)):
            start = list(KPList[i].values())[0]['start']
            end = list(KPList[i].values())[0]['end']
            call = requests.get(f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date={start}&end_date={end}&station={id}&product=predictions&datum=STND&time_zone=gmt&interval=hilo&units=english&application=Waves_data&format=json")
            response = call.json()['predictions']
            oneStep = [list(i.values()) for i in response]
            print(id, idName, start, end)
            [i.extend([id, idName]) for i in oneStep]
            if (i + 1) % 5 == 0:
                print("Buoy {} API call {} / {} complete, sleeping 45".format(id, i+1, len(keyList)))
                time.sleep(45)
        bigList.extend(oneStep)
    return bigList


pre_processed_data = initial_calls()

9444900 PortTownsend_WA 19000101 19051231
9444900 PortTownsend_WA 19060101 19091231
9444900 PortTownsend_WA 19100101 19151231
9444900 PortTownsend_WA 19160101 19191231
9444900 PortTownsend_WA 19200101 19251231
Buoy 9444900 API call 5 / 40 complete, sleeping 45
9444900 PortTownsend_WA 19260101 19291231
9444900 PortTownsend_WA 19300101 19351231
9444900 PortTownsend_WA 19360101 19391231
9444900 PortTownsend_WA 19400101 19451231
9444900 PortTownsend_WA 19460101 19491231
Buoy 9444900 API call 10 / 40 complete, sleeping 45


KeyboardInterrupt: 

In [207]:
len(historicInfill['dates'])

40

In [104]:
# Get list of keys from historicInfill
KPList = list(historicInfill.values())[0]
keyList = [list(i.keys())[0] for i in KPList]

    # Now make an api call using an f string to infill date range

#for iDen in tide_station_list:
    # To infill entire df is 40 time intervals * 28 stations 
    # id = iDen.values()
    # idName = iDen.keys()
    
def getHistoric():
    
    id = 9444900
    idName = 'PortTownsendWA'
    bigList = []
    for i in range(0, 1): #len(keyList)
        start = list(KPList[i].values())[0]['start']
        end = list(KPList[i].values())[0]['end']
        call = requests.get(f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date={start}&end_date={end}&station={id}&product=predictions&datum=STND&time_zone=gmt&interval=hilo&units=english&application=Waves_data&format=json")
        call2 = call.json()['predictions']
        oneStep = [list(i.values()) for i in call2]
        [i.append(idName) for i in oneStep]
        # bigList.append(oneStep)
        print(i)
    return oneStep



In [67]:
h = tideTest()

API call 5 / 40 complete
API call 10 / 40 complete
API call 15 / 40 complete
API call 20 / 40 complete
API call 25 / 40 complete
API call 30 / 40 complete
API call 35 / 40 complete
API call 40 / 40 complete
API call 5 / 40 complete
API call 10 / 40 complete
API call 15 / 40 complete
API call 20 / 40 complete
API call 25 / 40 complete
API call 30 / 40 complete
API call 35 / 40 complete
API call 40 / 40 complete


In [72]:
h

In [73]:
def tideTest():
    """ 
    Testing inputting one set of 200 years of data for one location.  Once I have that down, I can 
    infill my db automatically.

    """
    KPList = list(historicInfill.values())[0]
    keyList = [list(i.keys())[0] for i in KPList]
    station_list_keys = list(tide_station_list.keys())
    station_list_values = list(tide_station_list.values())
    bigList = []
    for j in range(0, 2):
        id = station_list_values[j]
        idName = station_list_keys[j]
        for i in range(len(keyList)):
            start = list(KPList[i].values())[0]['start']
            end = list(KPList[i].values())[0]['end']
            call = requests.get(f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date={start}&end_date={end}&station={id}&product=predictions&datum=STND&time_zone=gmt&interval=hilo&units=english&application=Waves_data&format=json")
            response = call.json()['predictions']
            oneStep = [list(i.values()) for i in response]
            [i.extend([id, idName]) for i in oneStep]
            if (i + 1) % 5 == 0:
                print("Buoy {} API call {} / {} complete".format(id, i+1, len(keyList)))
        bigList.extend(oneStep)
    return bigList

In [87]:
start = "19060101"
end = "19101231"
i = 9410170

call = requests.get(f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date={start}&end_date={end}&station={id}&product=predictions&datum=STND&time_zone=gmt&interval=hilo&units=english&application=Waves_data&format=json")
response = call.json()

In [88]:
response

{'error': {'message': ' The station is not a valid station or there is system error.'}}

In [6]:
a = tideTest()

iterator=0
for rows in a:
    # print(f"DATETIME:{rows[0]}, PRED:{rows[1]}, HL:{rows[2]}, LOC:{rows[3]}")

In [10]:
import csv

In [11]:
with open('test.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(a)

In [106]:
bl = getHistoric()

0


In [8]:
bl
bbl = []
# bbl = [i.append("word") for i in bl]
[print(i) for i in bl]

NameError: name 'bl' is not defined

In [ ]:
bl2

In [40]:
testCall = requests.get(f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date=19260101&end_date=19291231&station=9444900&product=predictions&datum=STND&time_zone=gmt&interval=hilo&units=english&application=Waves_data&format=json")

In [9]:
tc = testCall.json() #['predictions']  # [2].values() # [time , value, type]
tc
# 19400101", "end":"19451231
# [print(i) for i in tc]

NameError: name 'testCall' is not defined